In [4]:
from sqlalchemy import create_engine
import pandas as pd

In [14]:
engine = create_engine('sqlite:///sqlite_db_pythonsqlite.db')

In [15]:
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


TEST 

In [18]:
Bookings = pd.read_sql_query('SELECT * FROM Bookings', engine)
print(Bookings.head())

   bookid  facid  memid            starttime  slots
0       0      3      1  2012-07-03 11:00:00      2
1       1      4      1  2012-07-03 08:00:00      2
2       2      6      0  2012-07-03 18:00:00      2
3       3      7      1  2012-07-03 19:00:00      2
4       4      8      1  2012-07-03 10:00:00      1


In [19]:
Facilities = pd.read_sql_query('SELECT * FROM Facilities', engine)
print(Facilities.head())

   facid             name  membercost  guestcost  initialoutlay  \
0      0   Tennis Court 1         5.0       25.0          10000   
1      1   Tennis Court 2         5.0       25.0           8000   
2      2  Badminton Court         0.0       15.5           4000   
3      3     Table Tennis         0.0        5.0            320   
4      4   Massage Room 1         9.9       80.0           4000   

   monthlymaintenance  
0                 200  
1                 200  
2                  50  
3                  10  
4                3000  


In [20]:
Members = pd.read_sql_query('SELECT * FROM Members', engine)
print(Members.head())

   memid   surname firstname                       address  zipcode  \
0      0     GUEST     GUEST                         GUEST        0   
1      1     Smith    Darren    8 Bloomsbury Close, Boston     4321   
2      2     Smith     Tracy  8 Bloomsbury Close, New York     4321   
3      3    Rownam       Tim        23 Highway Way, Boston    23423   
4      4  Joplette    Janice    20 Crossing Road, New York      234   

        telephone recommendedby             joindate  
0  (000) 000-0000                2012-07-01 00:00:00  
1    555-555-5555                2012-07-02 12:02:05  
2    555-555-5555                2012-07-02 12:08:23  
3  (844) 693-0723                2012-07-03 09:32:15  
4  (833) 942-4710             1  2012-07-03 10:25:05  


In [21]:
q10 = pd.read_sql_query('''SELECT *
FROM (SELECT name, SUM(amt_paid) AS total_revenue
FROM (SELECT f.name, b.facid, b.bookid,
(CASE
 	WHEN b.memid = 0 THEN b.slots * f.guestcost
 	ELSE b.slots * f.membercost
 	END) AS amt_paid
FROM Facilities AS f
LEFT JOIN Bookings AS b
	ON f.facid = b.facid) AS bf
GROUP BY name) AS rev
WHERE total_revenue < 1000
ORDER BY total_revenue;''', engine)
print(q10.head())

            name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


In [22]:
q11 = pd.read_sql_query('''SELECT m1.surname AS mem_surname, m1.firstname AS mem_firstname, m1.recommendedby AS recommended_by_ID, (CASE WHEN m1.recommendedby > 0 THEN m2.surname END) AS recommendedby_surname, (CASE WHEN m1.recommendedby > 0 THEN m2.firstname END) AS recommendedby_firstname
FROM Members AS m1
LEFT JOIN Members AS m2
	ON m1.recommendedby = m2.memid
ORDER BY m1.surname, m2.firstname;''', engine)
print(q11.head())

  mem_surname mem_firstname recommended_by_ID recommendedby_surname  \
0       Bader      Florence                 9              Stibbons   
1       Baker       Timothy                13               Farrell   
2       Baker          Anne                 9              Stibbons   
3      Boothe           Tim                 3                Rownam   
4     Butters        Gerald                 1                 Smith   

  recommendedby_firstname  
0                  Ponder  
1                  Jemima  
2                  Ponder  
3                     Tim  
4                  Darren  


In [23]:
q12 = pd.read_sql_query('''SELECT b.facid, f.name, COUNT(b.memid) AS mem_usage
FROM (
SELECT facid, memid
FROM Bookings
WHERE memid !=0
) AS b
LEFT JOIN Facilities AS f ON b.facid = f.facid
GROUP BY b.facid;''', engine)
print(q12.head())

   facid             name  mem_usage
0      0   Tennis Court 1        308
1      1   Tennis Court 2        276
2      2  Badminton Court        344
3      3     Table Tennis        385
4      4   Massage Room 1        421


In [26]:
q13 = pd.read_sql_query('''SELECT b.months, COUNT( b.memid ) AS mem_usage
FROM (
SELECT strftime('%m', starttime ) AS months, memid
FROM Bookings
WHERE memid !=0
) AS b
GROUP BY b.months;''', engine)
print(q13.head())

  months  mem_usage
0     07        480
1     08       1168
2     09       1512
